# 03.11 - Working with Time Series

Pandas is able to work with several formats of date and time. We can categorize them as:

1. Time stamps 
2. Time intervals
3. Time deltas

### Dates and Times in Python

Let's have a look at what is already available in Python and their relationship with Pandas:

#### Native Python dates and times: <code>datetime</code> and <code>dateutil</code>

Using built-in <code>datetime</code>, we can manually build a date:

In [1]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

Using the <code>dateutil</code> module, you can parse dates from a variety of string formats:

In [2]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

Or printing the day of the week:

In [5]:
date.strftime('%A')

'Saturday'

### Typed arrays of times: NumPy's <code>datetime64</code>

The datetime64 dtype encodes dates as 64-bit integers, and thus allows arrays of dates to be represented very compactly. The datetime64 requires a very specific input format:

In [6]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

But we use vectorized operations on it:

In [7]:
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

The trade-off here is _time resolution_ vs. _maximum time span_. Whatever is our fundamental unit of choice (day, minute, second etc.) the range of encodable times is 2^64 times this fundamental unit.

In [8]:
#working in nanoseconds
np.datetime64('2015-07-04 12:59:59.50', 'ns')

numpy.datetime64('2015-07-04T12:59:59.500000000')

### Dates and times in pandas: best of both worlds

Pandas also provides a useful <code>Timestamp</code> object.  
From a group of these Timestamp objects, Pandas can construct a <code>DatetimeIndex</code> that can be used to index data in a Series or DataFrame.

In [9]:
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

In [10]:
date.strftime('%A')

'Saturday'

Additionally, we can do NumPy-style vectorized operations directly on this same object:

In [11]:
date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)